# Keras - Bidirectional LSTM 

In [1]:
from subprocess import check_output
print(check_output(["ls", "./"]).decode("utf8"))

glove.6B.50d.txt
sample_submission.csv
submission.csv
test.csv
Toxic_Prediction_with_keras.ipynb
Toxic_Prediction_with_keras_VERSION-01.ipynb
train.csv
weights_base.best.hdf5



In [2]:
import numpy as np
import pandas as pd

### Import keras Libs

In [3]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalAveragePooling1D, Dropout, GlobalMaxPool1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras import initializers, regularizers, constraints, optimizers, layers
import sys, os, re, csv, codecs

Using TensorFlow backend.


In [4]:
EMBEDDING_FILE = f'glove.6B.50d.txt'
TRAIN_DATA_FILE = f'train.csv'
TEST_DATA_FILE = f'test.csv'

In [5]:
max_features = 20000
maxlen = 100
embed_size = 50

In [6]:
train = pd.read_csv(TRAIN_DATA_FILE)

In [7]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
#train = train.sample(frac=1)
test = pd.read_csv(TEST_DATA_FILE)

In [9]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [10]:
list_sentences_train = train['comment_text'].fillna('New_Gen').values

In [28]:
#print(list_sentences_train)

In [12]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [13]:
list_classes

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [14]:
y = train[list_classes].values

In [15]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ..., 
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [26]:
y.shape

(159571, 6)

In [16]:
list_sentences_test = test['comment_text'].fillna('New_Gen').values

In [17]:
#print(list_sentences_test)

In [18]:
tokenizer = text.Tokenizer(num_words=max_features)

In [19]:
print(tokenizer)

In [20]:
tokenizer.fit_on_texts(list(list_sentences_train))

In [21]:
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [22]:
print(list_tokenized_train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [23]:
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [24]:
X_t.shape

(159571, 100)

In [25]:
X_te.shape

(153164, 100)

### Read Glove Word Vectors : word -> vector

In [31]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [32]:
all_embs = np.stack(embeddings_index.values())

In [33]:
emb_mean,emb_std = all_embs.mean(), all_embs.std()

In [34]:
emb_mean,emb_std

(0.020940498, 0.6441043)

In [35]:
word_index = tokenizer.word_index

In [36]:
print(word_index)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [37]:
nb_words = min(max_features, len(word_index))

In [38]:
nb_words

20000

In [39]:
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

In [40]:
embedding_matrix

array([[ 0.28230275,  0.11550656, -1.01240688, ...,  1.15092738,
        -1.68599883, -0.19232305],
       [-0.74396669, -0.0562929 , -0.51683096, ..., -0.5008556 ,
         0.20603953, -0.74239965],
       [-0.3852962 , -0.93124426,  0.08686534, ..., -0.39059653,
         0.08531565,  0.32133744],
       ..., 
       [ 0.09978704, -0.63971141, -1.23768621, ..., -0.45090129,
        -0.14872053, -0.30928202],
       [ 0.35951676,  1.35606281,  0.38097502, ..., -0.15871397,
         0.26675446, -0.21579909],
       [ 0.6584912 , -0.20792431,  1.56201039, ...,  0.27101193,
         0.70804951,  0.21238946]])

In [41]:
for word, i in word_index.items():
    if i >= max_features: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [42]:
embedding_matrix

array([[ 0.28230275,  0.11550656, -1.01240688, ...,  1.15092738,
        -1.68599883, -0.19232305],
       [ 0.41800001,  0.24968   , -0.41242   , ..., -0.18411   ,
        -0.11514   , -0.78580999],
       [ 0.68046999, -0.039263  ,  0.30186   , ..., -0.073297  ,
        -0.064699  , -0.26043999],
       ..., 
       [-0.13491   , -0.8635    , -0.033898  , ...,  0.29484999,
        -0.24315999,  0.81682003],
       [ 0.1178    ,  0.14624   , -0.28240001, ..., -0.19529   ,
        -0.13610999,  1.04270005],
       [-0.64388001, -0.54152   ,  0.10305   , ..., -0.06732   ,
        -0.73308003,  0.88625002]])

In [53]:
def get_model():
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [54]:
model = get_model()

In [55]:
# batch_size = 32
# epochs = 2

In [29]:
X_t.shape

(159571, 100)

In [30]:
y.shape

(159571, 6)

In [56]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 936s 7ms/step - loss: 0.0586 - acc: 0.9795 - val_loss: 0.0481 - val_acc: 0.9825
Epoch 2/2
143613/143613 [==============================] - 900s 6ms/step - loss: 0.0442 - acc: 0.9834 - val_loss: 0.0459 - val_acc: 0.9830


In [57]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 100, 50)           1000000   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 100, 100)          40400     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 306       
Total para

## Submission File Generate

In [58]:
y_test = model.predict([X_te], batch_size=1024, verbose=1)


153164/153164 [==============================] - 120s 783us/step


In [59]:
sample_submission = pd.read_csv(f'sample_submission.csv')


In [60]:
sample_submission[list_classes] = y_test
sample_submission.to_csv('submission.csv', index=False)